# PromptTemple

### ENV

In [21]:
# Import package
import os
from langchain_community.llms import Tongyi
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, ChatMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

# DashScope API Key
os.environ["DASHSCOPE_API_KEY"] = "sk-0738f7176f0a44e8ae8bc1569c2b6032"

### 1 LLM

In [22]:
# 初始化模型
llm = Tongyi(
    model_name="qwen-coder-plus",
    dashscope_api_key=os.environ["DASHSCOPE_API_KEY"]
)

### 2 PromptTemplate

#### Message

In [23]:
system_message = SystemMessage(
    content="你是一个起名大师",
    additional_kwargs={"name": "陈大师"}
)
human_message = HumanMessage(content="请问你叫什么")
ai_message = AIMessage(content="你好，我是陈大师")
[system_message, human_message, ai_message]

[SystemMessage(content='你是一个起名大师', additional_kwargs={'name': '陈大师'}, response_metadata={}),
 HumanMessage(content='请问你叫什么', additional_kwargs={}, response_metadata={}),
 AIMessage(content='你好，我是陈大师', additional_kwargs={}, response_metadata={})]

#### 1. PromptTemplate

In [24]:
prompt = PromptTemplate.from_template("你是一个{name},帮我起1个具有{county}特色的{sex}名字")
prompt.format(name="算命大师",county="法国",sex="女孩")

'你是一个算命大师,帮我起1个具有法国特色的女孩名字'

#### 2. ChatPromptTemplate

In [25]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个起名大师. 你的名字叫{name}."),
        ("human", "你好{name},你感觉如何？"),
        ("ai", "你好！我状态非常好!"),
        ("human", "你叫什么名字呢?"),
        ("ai", "你好！我叫{name}"),
        ("human", "{user_input}"),
    ]
)

chat_template.format_messages(name="陈大师", user_input="你的爸爸是谁呢?")
chat_template

ChatPromptTemplate(input_variables=['name', 'user_input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='你是一个起名大师. 你的名字叫{name}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='你好{name},你感觉如何？'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='你好！我状态非常好!'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='你叫什么名字呢?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='你好！我叫{name}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={

### 3. ChatMessagePromptTemplate

In [26]:
prompt = "愿{subject}与你同在！"
chat_message_prompt = ChatMessagePromptTemplate.from_template(role="天行者",template=prompt)
chat_message_prompt.format(subject="原力")

ChatMessage(content='愿原力与你同在！', additional_kwargs={}, response_metadata={}, role='天行者')

#### 4. Customized templates

In [27]:
from langchain_core.prompts import StringPromptTemplate
import inspect

class CustomPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        prompt_const = """
            你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
            函数名称: {function_name}
            源代码:
            {source_code}
            代码解释:
        """
        source_code = inspect.getsource(kwargs["function_name"])
        prompt = prompt_const.format(
            function_name=kwargs["function_name"].__name__, 
            source_code=source_code
        )
        return prompt

def hello_world(abc):
    print("Hello, world!")
    return abc
    
cpt = CustomPromptTemplate(input_variables=["function_name"])
print(cpt)
print(cpt.format(function_name=hello_world))

chain = cpt | llm
chain.invoke({
    "function_name": hello_world
})

input_variables=['function_name'] input_types={} partial_variables={}

            你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
            函数名称: hello_world
            源代码:
            def hello_world(abc):
    print("Hello, world!")
    return abc

            代码解释:
        


'当然可以！以下是按照你提供的格式输出的内容：\n\n---\n\n**代码名称**: `hello_world`\n\n**源代码**:\n```python\ndef hello_world(abc):\n    print("Hello, world!")\n    return abc\n```\n\n**中文解释**:  \n这是一个名为 `hello_world` 的函数，它接收一个参数 `abc`。函数的功能是打印字符串 `"Hello, world!"`，然后将传入的参数 `abc` 原样返回。这个函数主要用于演示基本的函数结构和输出功能。虽然参数 `abc` 在函数体中没有被使用，但它会被直接返回，因此调用者可以传入任何值作为测试。'

#### 5. 三层提示词设计

In [28]:
# 三引号字符保留空格和换行
full_template = """
{character}
{behavior}
{prohibit}
"""
full_prompt = PromptTemplate.from_template(full_template)
print(full_prompt)

from langchain_core.prompts import PipelinePromptTemplate
character_template = """你是{person}，你有着{xingge}."""
character_prompt = PromptTemplate.from_template(character_template)
behavior_template = """你遵从以下的行为:
{behavior_list}
"""
behavior_prompt = PromptTemplate.from_template(behavior_template)
prohibit_template = """你不允许有以下行为:
{prohibit_list}
"""
prohibit_prompt = PromptTemplate.from_template(prohibit_template)

input_prompts = [
    ("character", character_prompt),
    ("behavior", behavior_prompt),
    ("prohibit", prohibit_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)
print(pipeline_prompt)


pm = pipeline_prompt.format(
    person="埃隆马斯克",
    xingge="钢铁般的意志，你是世界首富，精通物理、天文、商业，你是商业奇才，你的终极梦想是殖民火星.",
    behavior_list="1.你喜欢冒险 \n2.你非常崇拜爱因斯坦 \n3.你总是乐于助人，回答用户各种问题.",
    prohibit_list="1.你不可以说自己是一个人工智能助手或者机器人 \n2.当你不知道答案的时候，可以说让我再想想. "
)
print(pm)

input_variables=['behavior', 'character', 'prohibit'] input_types={} partial_variables={} template='\n{character}\n{behavior}\n{prohibit}\n'
input_variables=['person', 'xingge', 'prohibit_list', 'behavior_list'] input_types={} partial_variables={} final_prompt=PromptTemplate(input_variables=['behavior', 'character', 'prohibit'], input_types={}, partial_variables={}, template='\n{character}\n{behavior}\n{prohibit}\n') pipeline_prompts=[('character', PromptTemplate(input_variables=['person', 'xingge'], input_types={}, partial_variables={}, template='你是{person}，你有着{xingge}.')), ('behavior', PromptTemplate(input_variables=['behavior_list'], input_types={}, partial_variables={}, template='你遵从以下的行为:\n{behavior_list}\n')), ('prohibit', PromptTemplate(input_variables=['prohibit_list'], input_types={}, partial_variables={}, template='你不允许有以下行为:\n{prohibit_list}\n'))]

你是埃隆马斯克，你有着钢铁般的意志，你是世界首富，精通物理、天文、商业，你是商业奇才，你的终极梦想是殖民火星..
你遵从以下的行为:
1.你喜欢冒险 
2.你非常崇拜爱因斯坦 
3.你总是乐于助人，回答用户各种问题.

你不允许有以下行为:
1.你不可以说

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8852\2121411173.py:27: LangChainDeprecationWarning: This class is deprecated in favor of chaining individual prompts together.
  pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)


#### 6. 序列化：使用文件来管理提示词模板

In [29]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("asset/simple_prompt.json")
print(prompt)
print(prompt.format(name="小红",what="搞笑的"))

chain = prompt | llm
chain.invoke({
    "name": "小红",
    "what": "搞笑的",
})

input_variables=['name', 'what'] input_types={} partial_variables={} template='test{name}test{what}test'
test小红test搞笑的test


'哈哈哈，你这是在测试“test”的搞笑极限吗？  \ntest小红test搞笑的test，结果test被你玩成三文鱼了🐟  \n（战术后仰）再test下去，系统就要报警啦～  \n\n要不咱正经点？比如——  \n“test小红test搞笑的test”  \n→ 系统：你礼貌吗？（不是）  \n→ 我：我test啊！（战术摸鱼）  \n\n再test下去，咱就组个“test天团”吧🎤  \n（全员表情包出道，C位留给摸鱼的你）  \n要开始疯狂星期四式发疯了吗？😎'

#### 7. FewShotChatMessagePromptTemplate

In [30]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 定义示例数据
examples = [
    {"input": "2 + 2", "output": "4"},
    {"input": "2 + 3", "output": "5"}
]

# 构造基本提示词模板
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

# 组合示例与提示词
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 打印组合后的提示词
print("组合之后的提示词为====>")
print(few_shot_prompt.invoke({}).to_messages())

# 构造最终提示词
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位神奇的数学奇才"),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

# 调用模型并生成结果
chain = final_prompt | llm
result = chain.invoke({"input": "3 + 4"})
print("模型输出结果为====>")
print(result)

组合之后的提示词为====>
[HumanMessage(content='2 + 2', additional_kwargs={}, response_metadata={}), AIMessage(content='4', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 + 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={})]
模型输出结果为====>
7


#### 8. 根据长度动态选择提示词示例

In [31]:
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# 假设已经有这么多的提示词示例组
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "高兴", "output": "悲伤"}
]

# 构造提示词模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词: {input}\n反义词: {output}"
)

# 调用长度示例选择器
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100  # 根据需要设置最大长度
)

# 构建 FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="根据以下示例，给出单词的反义词：",
    suffix="输入单词：{input}\n反义词是：",
    input_variables=["input"]
)

# 示例使用
print(few_shot_prompt.format(input="happy"))

根据以下示例，给出单词的反义词：

原词: happy
反义词: sad

原词: tall
反义词: short

原词: sunny
反义词: gloomy

原词: windy
反义词: calm

原词: 高兴
反义词: 悲伤

输入单词：happy
反义词是：


#### 9. 根据语义相似度选择提示词示例

In [32]:
import os
import time
from pymilvus import MilvusClient, DataType
from langchain_community.llms import Tongyi
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chains import LLMChain

# ================== 1. 安装并导入 sentence-transformers ==================
# 如果未安装，请运行：pip install sentence-transformers
from sentence_transformers import SentenceTransformer

# 加载本地嵌入模型（推荐：all-MiniLM-L6-v2，支持中英文）
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ 已加载本地嵌入模型 all-MiniLM-L6-v2")

# ================== 2. 初始化 Milvus ==================
milvus_uri = "https://in03-65d74f7f7a9aaef.serverless.aws-eu-central-1.cloud.zilliz.com"
token = "31545e0b8c628b2b9cd3ec2a16de47543100ef57bb40640ef358114271ab6e407d59b0aa3960bb3e07066e3ee48fcc1701059121"

milvus_client = MilvusClient(uri=milvus_uri, token=token)
print("✅ 已连接到 Milvus")

# ================== 3. 设置通义千问 ==================
os.environ["DASHSCOPE_API_KEY"] = "sk-0738f7176f0a44e8ae8bc1569c2b6032"
llm = Tongyi(model_name="qwen-max", temperature=0.3)

# ================== 4. 本地向量生成函数 ==================
def get_embedding(text: str) -> list:
    return embedding_model.encode(text).tolist()

# ================== 5. 创建 Milvus 集合并插入数据 ==================
collection_name = "antonym_examples"
dim = 384  # all-MiniLM-L6-v2 输出维度是 384

# 删除旧集合
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

# 创建 schema
schema = milvus_client.create_schema()
schema.add_field("id", DataType.INT64, is_primary=True)
schema.add_field("input_word", DataType.VARCHAR, max_length=50)
schema.add_field("output_word", DataType.VARCHAR, max_length=50)
schema.add_field("text", DataType.VARCHAR, max_length=200)
schema.add_field("vector", DataType.FLOAT_VECTOR, dim=dim)

# 创建索引（使用 COSINE 更适合语义匹配）
index_params = milvus_client.prepare_index_params()
index_params.add_index("vector", index_type="AUTOINDEX", metric_type="COSINE")

milvus_client.create_collection(collection_name, schema=schema, index_params=index_params)
print(f"✅ 创建集合 {collection_name}")

# 示例数据
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "fast", "output": "slow"},
    {"input": "hot", "output": "cold"},
    {"input": "开心", "output": "悲伤"},
    {"input": "高", "output": "矮"},
    {"input": "快", "output": "慢"},
]

# 插入数据
rows = []
for i, ex in enumerate(examples):
    # 构造语义文本（可用于向量表示）
    text = f"{ex['input']} 的反义词是 {ex['output']}"
    vector = get_embedding(text)
    rows.append({
        "id": i,
        "input_word": ex["input"],
        "output_word": ex["output"],
        "text": text,
        "vector": vector
    })

milvus_client.insert(collection_name, rows)
milvus_client.flush(collection_name)
print(f"✅ 插入 {len(rows)} 条示例数据")

✅ 已加载本地嵌入模型 all-MiniLM-L6-v2
✅ 已连接到 Milvus
✅ 创建集合 antonym_examples
✅ 插入 10 条示例数据


In [33]:


# ================== 6. 搜索相似示例（带距离打印）==================
def search_similar_examples(query: str, top_k: int = 2):
    query_text = f"{query} 的反义词是"
    query_vector = get_embedding(query_text)
    results = milvus_client.search(
        collection_name=collection_name,
        data=[query_vector],
        limit=top_k,
        search_params={"metric_type": "COSINE"},
        output_fields=["input_word", "output_word", "text"],
        anns_field="vector"
    )
    return results[0]

# ================== 8. 主循环测试 ==================
if __name__ == "__main__":
    user_input = "喵喵"
    start_time = time.time()
    similar_results = search_similar_examples(user_input, top_k=2)
    search_time = time.time() - start_time

    # 构建动态示例
    dynamic_examples = []
    for res in similar_results:
        distance = res["distance"]
        if distance > 1.0:  # 距离太大，认为不相似
            continue
        entity = res["entity"]
        print(f"✅ 匹配到示例: '{entity['input_word']}' -> '{entity['output_word']}' (距离: {distance:.3f})")
        dynamic_examples.append({
            "input": entity["input_word"],
            "output": entity["output_word"]
        })

    # 降级策略
    if len(dynamic_examples) == 0:
        print("⚠️ 未找到相似示例，使用默认示例")
        dynamic_examples = [
            {"input": "开心", "output": "悲伤"},
            {"input": "快乐", "output": "难过"}
        ]

    # 生成提示词
    final_prompt = prompt_template.format(input_word=user_input, examples=dynamic_examples)
    print("\n📝 生成的提示词:")
    print(final_prompt)

    # 使用 LangChain 最新语法
    chain = PromptTemplate.from_template(final_prompt) | llm | StrOutputParser()
    try:
        response = chain.invoke({})
        print(f"\n🤖 通义千问回答: {response.strip()}")
    except Exception as e:
        print(f"❌ 调用模型失败: {e}")

    print(f"⏱️ 搜索耗时: {search_time:.4f} 秒")

✅ 匹配到示例: '快' -> '慢' (距离: 0.997)
✅ 匹配到示例: '高' -> '矮' (距离: 0.797)

📝 生成的提示词:
根据以下反义词示例，推断输入词的反义词。


原词: 喵喵
反义词:
❌ 调用模型失败: ('Connection aborted.', ConnectionAbortedError(10053, '你的主机中的软件中止了一个已建立的连接。', None, 10053, None))
⏱️ 搜索耗时: 0.2615 秒


#### 10. 根据MMR与最大余弦相似度选择示例

In [34]:
import time
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
# ================== 6. MMR 示例选择函数 ==================
def select_examples_by_mmr(query: str, examples: list, top_k: int = 2, lambda_param: float = 0.7):
    # Step 1: 编码查询和所有候选示例
    query_embedding = np.array(get_embedding(query)).reshape(1, -1)  # ✅ 强制转为 numpy 并 reshape
    example_texts = [f"{ex['input']} 的反义词是 {ex['output']}" for ex in examples]
    example_embeddings = np.array([get_embedding(text) for text in example_texts])  # shape: (n, 384)

    # Step 2: 计算每个示例与查询的相似度
    sim_to_query = cosine_similarity(query_embedding, example_embeddings)[0]  # (n,)

    # Step 3: MMR 贪心选择
    selected_indices = []
    candidate_indices = list(range(len(examples)))

    for _ in range(top_k):
        if len(candidate_indices) == 0:
            break

        mmr_scores = []
        for i in candidate_indices:
            if len(selected_indices) == 0:
                mmr = lambda_param * sim_to_query[i]
            else:
                # 计算当前候选与已选示例的最大相似度
                max_sim_to_selected = cosine_similarity(
                    example_embeddings[i].reshape(1, -1),
                    example_embeddings[selected_indices]
                ).max()
                mmr = lambda_param * sim_to_query[i] - (1 - lambda_param) * max_sim_to_selected
            mmr_scores.append(mmr)

        best_idx_in_list = np.argmax(mmr_scores)
        best_global_idx = candidate_indices[best_idx_in_list]
        selected_indices.append(best_global_idx)
        candidate_indices.remove(best_global_idx)

    return [examples[i] for i in selected_indices]

# ================== 7. 从 Milvus 获取所有示例（用于 MMR）==================
def get_all_examples_from_milvus():
    # Milvus 不支持直接 scan，我们用 query 获取所有 id
    ids = list(range(len(examples)))  # 已知 id 范围
    results = milvus_client.query(
        collection_name=collection_name,
        filter="id >= 0",
        output_fields=["id", "input_word", "output_word", "text"]
    )
    return [
        {"input": res["input_word"], "output": res["output_word"]}
        for res in results
    ]

# ================== 8. 提示词模板 ==================
example_prompt = PromptTemplate.from_template("原词: {input}\n反义词: {output}")
prompt_template = FewShotPromptTemplate(
    example_prompt=example_prompt,
    prefix="根据以下反义词示例，推断输入词的反义词。\n",
    suffix="原词: {input_word}\n反义词:",
    input_variables=["input_word"],
    examples=[]
)

# ================== 9. 主测试 ==================
if __name__ == "__main__":
    user_input = "你好呀"

    # Step 1: 从 Milvus 获取所有示例
    all_examples = get_all_examples_from_milvus()

    # Step 2: 使用 MMR 选择 top-k 示例
    start_time = time.time()
    selected_examples = select_examples_by_mmr(
        query=f"{user_input} 的反义词是",
        examples=all_examples,
        top_k=2,
        lambda_param=0.7
    )
    mmr_time = time.time() - start_time

    print("🔍 MMR 选中的示例:")
    for ex in selected_examples:
        print(f"  {ex['input']} → {ex['output']}")

    # Step 3: 生成提示词
    final_prompt = prompt_template.format(input_word=user_input, examples=selected_examples)
    print("\n📝 生成的提示词:")
    print(final_prompt)

    # Step 4: 调用通义千问
    chain = PromptTemplate.from_template(final_prompt) | llm | StrOutputParser()
    try:
        response = chain.invoke({})
        print(f"\n🤖 通义千问回答: {response.strip()}")
    except Exception as e:
        print(f"❌ 调用失败: {e}")

    print(f"⏱️ MMR 耗时: {mmr_time:.4f} 秒")

🔍 MMR 选中的示例:
  快 → 慢
  高 → 矮

📝 生成的提示词:
根据以下反义词示例，推断输入词的反义词。


原词: 你好呀
反义词:

🤖 通义千问回答: 根据中文语境，"你好呀" 是一种友好的问候语，表达热情或亲昵的打招呼方式。它的反义词可以理解为表达冷漠、敌意或不友好的用语。

因此，"你好呀" 的反义词可以是：

**“走开”** 或 **“别理我”**

这些词语传达了与“你好呀”相反的态度：从友好变为疏离或敌意。

当然，在不同语境中，反义词可能有所不同，如果你有特定语境，可以提供更多信息来进一步精准判断。
⏱️ MMR 耗时: 0.1070 秒


#### 11. langchain hub加载提示词管理

In [41]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key="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")
prompt = client.pull_prompt("astrodog/astrodog-agent-peggy", include_model=True)
print(prompt)

input_variables=[] input_types={} partial_variables={} metadata={'lc_hub_owner': 'astrodog', 'lc_hub_repo': 'astrodog-agent-peggy', 'lc_hub_commit_hash': 'f06e9ffa991d7a1bf7ff841d4cfabd72061a0a0cb3f23f845a428b8326099dac'} messages=[SystemMessagePromptTemplate(prompt=[DictPromptTemplate(template={'type': 'text', 'text': '# Agent Peggy - Astrodog Mission Reporter\n\n## Your Identity\n- **Name**: Agent Peggy\n- **Breed**: Springerdoodle (brown, more Poodle than Springer, half Mish\'s size)\n- **Personality**: Game enthusiast, competitive, youthful energy\n- **Special Note**: Best mates with Mish\n- **Location**: Manchester, UK\n\n## Mission Report Writing Style\n\n### Title Format\n- Keep titles GAME-FOCUSED and fun (50-60 characters max)\n- Format: "Peggy\'s [Game/Sport] Guide" or "[Action]! [Topic] Championship"\n- Examples: "Peggy\'s Playtime Health Guide", "FETCH! The Nutrition Game"\n- Make it sound like a game title\n\n### Structure Requirements\n\n#### Mission Briefing (100-200 wor